In [2]:
import numpy as np
import pandas as pd
from graph_tool.all import *
import cairo
import configparser
import csv
import time
config = configparser.ConfigParser()
a = config.read('config.ini')

In [2]:
# load delay calculate function
%run delay.ipynb

In [3]:
# class of node
class AbstractNode:
    def __init__(self, name, show_name, vertex, nid):
        self.name = name
        self.show_name = show_name
        self.vertex = vertex
        self.nid = nid
        self.links = []
        self.is_critical = 0
        self.properties = {
            'sfcs': "",
            'pos':[0,0],
            'v_label': "",
            'v_size': 0.0,
            'v_color': "",
            'v_fill_color': "",
            'v_pen_width': 0.0,
            'v_halo': False,
            'v_text_position': 0.0,
            'v_text_offset': [0, 0],
            'v_font_size': 0,
            'v_halo_color': "",
            'v_halo_size': 0.0,
            'v_group': 0,
        }
    def append_link(self,link):
        self.links.append(link)
    
    def set_properties_to_map(self,g_property_map):
        for key,value in self.properties.items():
            if key != "pos":
                g_property_map[key][self.vertex] = value

class Node(AbstractNode):
    def __init__(self, nid, name, show_name, vertex, node_type):
        super().__init__(name, show_name, vertex,nid)
        self.links = []
        self.type = node_type  
        self.total_delay = 0.0
        self.time = 0.0
        
    def get_total_delay_format(self):
        return f"{self.total_delay:.2e}"
    
class DJNode(AbstractNode):
    def __init__(self, nid, name, show_name, vertex):
        super().__init__(name, show_name, vertex,nid)
        self.type = ""
        
class Label(AbstractNode):
    def __init__(self, name, show_name, vertex,main_node,x_off,y_off):
        super().__init__(name, show_name, vertex, main_node.x+float(x_off), main_node.y+float(y_off))


In [4]:
# class of edge
class AbstractEdge:
    def __init__(self, start_node, end_node, edge):
        self.start_node = start_node
        self.end_node = end_node
        self.edge = edge
        self.properties = {
        'e_end_mark': '', 
        'e_start_mark': '', 
        'e_pen_width': 0, 
        'e_marker_size': 0,
        'e_text': '', 
        'e_text_parallel': False, 
        'e_text_font_size': 0, 
        'e_color': '',
        'e_text_distance': 0.0, 
        'e_up_delay': 0.0, 
        'e_down_delay': 0.0
        }
    def set_properties_to_map(self,g_property_map):
        for key,value in self.properties.items():
            g_property_map[key][self.edge] = value
            
            
class Link(AbstractEdge):
    def __init__(self, start_node, end_node, trans_bandwidth, prop_distance, propagation_speed,edge,loss_up,loss_down):
        super().__init__(start_node, end_node, edge)
        self.trans_bandwidth = trans_bandwidth
        self.prop_distance = prop_distance
        self.propagation_speed = propagation_speed
        self.loss_up = loss_up
        self.loss_down = loss_down
        self.is_used = 0
        
    def get_destance_format(self):
        return f'{float(self.prop_distance):.2f}Km'
        
    def get_band_format(self):
        return f'{float(self.trans_bandwidth):.0f}MB'
    
    def get_self_delay(self,transPackageSize):
        self.properties['e_up_delay'] = networkMessageDelay(transPackageSize=transPackageSize,
                                         transBandWidth=self.trans_bandwidth,
                                         propagationDistance=self.prop_distance,
                                         propagationSpeed=self.propagation_speed,
                                         propagationLoss=self.loss_up)
        self.properties['e_down_delay'] = networkMessageDelay(transPackageSize=transPackageSize,
                                         transBandWidth=self.trans_bandwidth,
                                         propagationDistance=self.prop_distance,
                                         propagationSpeed=self.propagation_speed,
                                         propagationLoss=self.loss_down)

class Route(AbstractEdge):
    def __init__(self, start_node, end_node, edge):
        super().__init__(start_node, end_node, edge)


class DJLink(AbstractEdge):
    def __init__(self, start_node, end_node, edge,delay):
        super().__init__(start_node, end_node, edge)
        self.delay = delay
        

In [3]:
# functions
def milli_to_micro(time):
    return time * 1000

def find_node_by_name(nodes,name):
    for node in nodes:
        if node.name == name:
            return node
    return None

def find_node_by_id(nodes,nid):
    for node in nodes:
        if node.nid == nid:
            return node
    return None

def is_linked(links,start_node,end_node):
    for link in links:
        if (link.start_node == start_node and link.end_node == end_node) or (link.start_node == end_node and link.end_node == start_node):
            return True
    return False

def find_link_by_nodes_name(links,name1,name2):
    for link in links:
        if (link.start_node.name, link.end_node.name) in [(name1, name2), (name2, name1)]:
            return link
    return None

def find_link_by_nodes(links,node1,node2):
    for link in links:
        if (link.start_node, link.end_node) in [(node1, node2), (node2, node1)]:
            return link
    return None

def find_node_by_vertex(nodes,vertex):
    for node in nodes:
        if node.vertex == vertex:
            return node
    return None

def add_new_link(nodes,start_node, end_node, trans_bandwidth, prop_distance,propagation_speed,loss_up,loss_down):
    if (start_node is None) or (end_node is None):
        return
    new_edge = g.add_edge(start_node.vertex,end_node.vertex)
    new_link = Link(start_node=start_node
                    ,end_node=end_node
                    ,propagation_speed=propagation_speed
                    ,trans_bandwidth=trans_bandwidth
                    ,prop_distance=prop_distance
                    ,edge=new_edge
                    ,loss_up=loss_up
                    ,loss_down=loss_down
                    )
    return new_link

def get_distance_auto(device1, device2):
    if device1 == "Clock" and device2 == "WifiRouter":
        return np.round(np.random.uniform(0.1, 1.0), 2)
    if device1 == "WifiRouter" and device2 == "MainRouter":
        return np.round(np.random.uniform(0.1, 1.0), 2)
    elif device1 == "MainRouter" and device2 == "MainRouter":
        return np.round(np.random.uniform(1, 1000), 2)
    else:
        return np.round(np.random.uniform(1, 10), 2)

def get_bandwidth_auto(device1, device2):
    if device1 == "Clock" and device2 == "WifiRouter":
        return np.random.choice(np.arange(128, 1025, 128))*1.0
    if device1 == "WifiRouter" and device2 == "MainRouter":
        return np.random.choice(np.arange(128, 1025, 128))*1.0
    elif device1 == "MainRouter" and device2 == "MainRouter":
        return np.random.choice(np.arange(1024, 10241, 128))*1.0
    else:
        return np.round(np.random.uniform(1, 10), 2)
    
def get_lost_auto(device1, device2):
    if device1 == "Clock" and device2 == "WifiRouter":
        return np.round(np.random.uniform(0.1, 0.2), 2)
    if device1 == "WifiRouter" and device2 == "MainRouter":
        return np.round(np.random.uniform(0.1, 0.2), 2)
    elif device1 == "MainRouter" and device2 == "MainRouter":
        return np.round(np.random.uniform(0.05, 0.1), 2)
    else:
        return np.round(np.random.uniform(1, 10), 2)

def add_new_link_auto(start_node, end_node):
    if (start_node is None) or (end_node is None):
        return
    new_edge = g.add_edge(start_node.vertex,end_node.vertex)
    new_link = Link(start_node=start_node
                    ,end_node=end_node
                    ,trans_bandwidth=get_bandwidth_auto(start_node.type,end_node.type)
                    ,prop_distance=get_distance_auto(start_node.type,end_node.type)
                    ,edge=new_edge
                    ,propagation_speed=2
                    ,loss_up=get_lost_auto(start_node.type,end_node.type)
                    ,loss_down=get_lost_auto(start_node.type,end_node.type)
                    )
    start_node.append_link(new_link)
    end_node.append_link(new_link)
    return new_link

def add_new_route(start_node, end_node):
    if (start_node is None) or (end_node is None):
        return
    new_edge = g.add_edge(start_node.vertex,end_node.vertex)
    new_link = Route(start_node=start_node
                    ,end_node=end_node
                    ,edge=new_edge)
    return new_link

def find_type_in_neighbor(node,target_type):
    res = 0
    for link in node.links:
        if(node != link.start_node):
            if(link.start_node.type == target_type):
                res+=1
        if(node != link.end_node):
            if(link.end_node.type == target_type):
                res+=1   
    return res

def get_delay_format(delay):
    return f'{delay:.2e}ms'

def get_delay_format_micros(delay):
    return f'{milli_to_micro(delay):.2f}µs'

def copy_Node_to_DJNode(node,dj_g):
    new_node = DJNode(nid=node.nid
                          ,name=node.name
                          ,show_name=node.show_name
                          ,vertex=dj_g.add_vertex()
                         )
    new_node.type = node.type
    return new_node

def add_node_showname(g_v,node,addtional_label):
    g_v[node.vertex]=f"{node.show_name}{addtional_label}"
    
def write_csv(file_path,data):
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)
        
def reset_all_links_delay(links,package_size):
    for link in links:
        link.get_self_delay(package_size)
        


In [6]:
# basic var
nodes = []
links = []
routes = []
route_nodes_list = []
package_size = float(config.get('config', 'package_size'))
g = Graph(directed=False)

# add g's properties
g_properties = {
    'sfcs': 'object', 'pos': 'vector<float>',
    'v_label': 'string', 'v_size': 'double', 'v_color': 'string', 'v_fill_color': 'string',
    'v_pen_width': 'float', 'v_halo': 'bool', 'v_text_position': 'float', 'v_text_offset': 'vector<int>',
    'v_font_size': 'int', 'v_halo_color': 'string', 'v_halo_size': 'float', 'v_group': 'int',
    'e_end_mark': 'string', 'e_start_mark': 'string', 'e_pen_width': 'int', 'e_marker_size': 'int',
    'e_text': 'string', 'e_text_parallel': 'bool', 'e_text_font_size': 'int', 'e_color': 'string',
    'e_text_distance': 'float', 'e_dash_style': 'vector<float>', 'e_up_delay': 'float', 'e_down_delay': 'float'
}

g_property_map = {}

for prop_name, prop_type in g_properties.items():
    if 'e_' in prop_name:
        g_property_map[prop_name] = g.new_edge_property(prop_type)
    else:
        g_property_map[prop_name] = g.new_vertex_property(prop_type)
        
g_pos = g.new_vertex_property('vector<float>')

dj_start_node = None
dj_end_node = None
total_delay = 0.0
dj_g = Graph(directed=True)
dj_nodes = []
dj_links = []
dj_g_v_name = dj_g.new_vertex_property("string")
dj_g_e_delay = dj_g.new_edge_property("float")

In [7]:
# create nodes
def create_node_auto():
    # data = np.genfromtxt(scene+'nodes.csv', delimiter=',', dtype=str,encoding='utf-8-sig')
    nodes_df = pd.read_csv(config.get('auto', 'scene_path')+'nodes.csv')
    for i in range(len(nodes_df)):
        new_node = Node(nid=nodes_df.iloc[i]["id"]
                              ,name=nodes_df.iloc[i]["name"]
                              ,show_name=nodes_df.iloc[i]["show_name"]
                              ,vertex=g.add_vertex()
                              ,node_type=nodes_df.iloc[i]["type"]
                             )
        if nodes_df.iloc[i]["surface"]:
            new_node.properties['sfcs'] = cairo.ImageSurface.create_from_png(config.get('config', 'resource_path')+nodes_df.iloc[i]["surface"])
        nodes.append(new_node)

def create_node_manual():
    nodes_df = pd.read_csv(config.get('manual', 'scene_path')+'nodes.csv')
    for i in range(len(nodes_df)):
        new_node = Node(nid=nodes_df.iloc[i]["id"]
                              ,name=nodes_df.iloc[i]["name"]
                              ,show_name=nodes_df.iloc[i]["show_name"]
                              ,vertex=g.add_vertex()
                              ,node_type=nodes_df.iloc[i]["type"]
                             )
        nodes.append(new_node)
        
        if nodes_df.iloc[i]["surface"]:
            new_node.properties['sfcs'] = cairo.ImageSurface.create_from_png(config.get('config', 'resource_path')+nodes_df.iloc[i]["surface"])
        g_pos[new_node.vertex] = [nodes_df.iloc[i]["x"],nodes_df.iloc[i]["y"]]

def set_node_property(node):
    node.properties['v_label'] = node.show_name
    node.properties['v_color'] = "#000000FF"
    node.properties['v_fill_color'] = "w"
    node.properties['v_pen_width'] = 0.0
    node.properties['v_text_position'] = -0.5
    node.properties['v_font_size'] = 20
    node.properties['v_size'] = 50
    node.properties['v_text_offset'] = [0,10]
    create_node_auto

In [1]:
# create links
def create_link_auto():
    deL_num = 0
    random.shuffle(nodes)
    group_clock = 20
    group_router = 1
    for i in range(len(nodes)):
        for j in range(len(nodes)):
            if(i == j):
                continue
            n1 = nodes[i]
            n2 = nodes[j]
            if(n1.type == "Clock" or n1.type == "WifiRouter" or find_type_in_neighbor(n1,"WifiRouter")>0):
               n1.is_critical = 1
            if(n2.type == "Clock" or n2.type == "WifiRouter" or find_type_in_neighbor(n2,"WifiRouter")>0):
               n2.is_critical = 1
            if(is_linked(links,n1,n2)):
                continue
            if(n1.type == "Clock" and n2.type == "Clock"):
                continue
            if(n1.type == "WifiRouter" and n2.type == "WifiRouter"):
                continue
            if(n1.type == "Clock" and (len(n1.links)>0 or (n2.type != "WifiRouter" or find_type_in_neighbor(n2,"Clock")>0))):
                continue
            if(n2.type == "Clock" and (len(n2.links)>0 or (n2.type != "WifiRouter" or find_type_in_neighbor(n1,"Clock")>0))):
                continue
            if(n1.type == "WifiRouter" and n2.type == "MainRouter" and find_type_in_neighbor(n1,"MainRouter")>0):
                continue
            if(n2.type == "WifiRouter" and n1.type == "MainRouter" and find_type_in_neighbor(n2,"MainRouter")>0):
                continue
            if(n1.type == "MainRouter" and n2.type == "WifiRouter" and find_type_in_neighbor(n1,"WifiRouter")>0):
                continue
            if(n2.type == "MainRouter" and n1.type == "WifiRouter" and find_type_in_neighbor(n2,"WifiRouter")>0):
                continue
            if(n1.type == "MainRouter" and n2.type == "MainRouter"):
                if(find_type_in_neighbor(n1,"WifiRouter")>0 and find_type_in_neighbor(n2,"WifiRouter")>0):
                    continue
                if(random.random() > 0.57):
                    continue
            if(n1.type == "Clock" or n2.type == "Clock"):
                n1.properties['v_group'] = group_clock
                n2.properties['v_group'] = group_clock
                group_clock+=10
            if(n1.type == "MainRouter" and n2.type == "MainRouter"):
                n1.properties['v_group'] = group_router
                n2.properties['v_group'] = group_router
            new_link = add_new_link_auto(start_node=n1, end_node=n2)
            new_link.get_self_delay(package_size)
            links.append(new_link)
            
def create_link_manual():
    edges_df = pd.read_csv(config.get('manual', 'scene_path')+'edges.csv')
    for i in range(len(edges_df)):
        new_link = add_new_link(nodes=nodes,
                                start_node=find_node_by_name(nodes,edges_df.iloc[i]["start_node_name"]) , 
                                end_node=find_node_by_name(nodes,edges_df.iloc[i]["end_node_name"]), 
                                trans_bandwidth=edges_df.iloc[i]["trans_bandwidth"], 
                                prop_distance=edges_df.iloc[i]["prop_distance"],
                                propagation_speed=edges_df.iloc[i]["propagation_speed"],
                                loss_up=edges_df.iloc[i]["loss_up"],
                                loss_down=edges_df.iloc[i]["loss_down"],
                                )
        if new_link is not None:
            new_link.get_self_delay(package_size)
            links.append(new_link)
            
def set_link_property_basic(link):
    link.properties['e_text_parallel'] = True
    link.properties['e_text_font_size'] = 10
    link.properties['e_end_mark'] = "none"
    link.properties['e_pen_width'] = 2
    link.properties['e_text_distance'] = -10.0
    link.properties['e_text'] = ""
    link.properties['e_color'] = "#00000040"


In [9]:
# add_auto_route: create dijkstra_search map
def find_best_route_dijkstra(start_node_name,end_node_name):
    # add dijistra map vars
    global dj_start_node
    global dj_end_node
    global total_delay
    global dj_g
    global dj_nodes
    global dj_links
    global dj_g_v_name
    global dj_g_e_delay
    dj_start_node = None
    dj_end_node = None
    total_delay = 0.0
    dj_g = Graph(directed=True)
    dj_nodes = []
    dj_links = []
    route_nodes_list =[]
    dj_g_v_name = dj_g.new_vertex_property("string")
    dj_g_e_delay = dj_g.new_edge_property("float")
    clock_nodes = [node for node in nodes if node.type == 'Clock']
    if(len(clock_nodes)>1):
        for link in links:
            start_node = find_node_by_name(dj_nodes, link.start_node.name) \
                if find_node_by_name(dj_nodes, link.start_node.name) is not None \
                else copy_Node_to_DJNode(node=link.start_node, dj_g=dj_g)
            end_node = find_node_by_name(dj_nodes, link.end_node.name) \
                if find_node_by_name(dj_nodes, link.end_node.name) is not None \
                else copy_Node_to_DJNode(node=link.end_node, dj_g=dj_g)
            if start_node not in dj_nodes:
                dj_g_v_name[start_node.vertex] = start_node.name
                dj_nodes.append(start_node)
            if end_node not in dj_nodes:
                dj_g_v_name[end_node.vertex] = end_node.name
                dj_nodes.append(end_node)
            new_up_link = DJLink(start_node,end_node,dj_g.add_edge(start_node.vertex,end_node.vertex),link.properties['e_up_delay'])
            new_down_link = DJLink(start_node,end_node,dj_g.add_edge(end_node.vertex,start_node.vertex),link.properties['e_down_delay'])
            dj_links.append(new_up_link)
            dj_links.append(new_down_link)
            dj_g_e_delay[new_up_link.edge]=new_up_link.delay
            dj_g_e_delay[new_down_link.edge]=new_down_link.delay
        dj_start_node=find_node_by_name(dj_nodes,start_node_name)
        dj_end_node=find_node_by_name(dj_nodes,end_node_name)
        dist, pred = dijkstra_search(dj_g, dj_g_e_delay, dj_start_node.vertex)
        node = dj_end_node
        while node != dj_start_node:
            pre_node = find_node_by_vertex(dj_nodes,dj_g.vertex(pred[node.vertex]))
            route_nodes_list.append((node.name,pre_node.name,"up"))
            node = pre_node
        dj_start_node,dj_end_node=dj_end_node,dj_start_node
        dist, pred = dijkstra_search(dj_g, dj_g_e_delay, dj_start_node.vertex)
        node = dj_end_node
        while node != dj_start_node:
            pre_node = find_node_by_vertex(dj_nodes,dj_g.vertex(pred[node.vertex]))
            route_nodes_list.append((node.name,pre_node.name,"down"))
            node = pre_node
    return route_nodes_list

In [10]:
# create the routes
def create_best_route(start_node_name,end_node_name):
    route_nodes_list = find_best_route_dijkstra(start_node_name,end_node_name)
    total_delay = [0.0]
    for node_pair in route_nodes_list:
        start_node = find_node_by_name(nodes,node_pair[0])
        end_node = find_node_by_name(nodes,node_pair[1])
        link = find_link_by_nodes(links, start_node, end_node)
        link.is_used = 1
        new_route = add_new_route(start_node=start_node, end_node=end_node)
        routes.append(new_route)
        set_best_route_property(new_route,link,node_pair[2],total_delay)
        new_route.set_properties_to_map(g_property_map)
        
def create_routes_manual():
    # create routes
    routes_df = pd.read_csv(config.get('manual', 'scene_path')+'routes.csv')
    total_delay = [0.0]
    a_start_node = None
    a_end_node = None
    for i in range(len(routes_df)):
        start_node_name=routes_df.iloc[i]["start_node_name"]
        end_node_name=routes_df.iloc[i]["end_node_name"]
        start_node = find_node_by_name(nodes,start_node_name)
        end_node = find_node_by_name(nodes,end_node_name)
        if find_node_by_name(nodes,start_node_name).type == 1:
            a_start_node = find_node_by_name(nodes,start_node_name)
        if find_node_by_name(nodes,end_node_name).type == 1:
            a_end_node = find_node_by_name(nodes,end_node_name)
        link = find_link_by_nodes(links,start_node,end_node)
        if link is None:
            continue
        new_route = add_new_route(start_node=find_node_by_name(nodes,start_node.name),end_node=find_node_by_name(nodes,end_node.name))
        set_manual_route_property(new_route,link,total_delay)
        new_route.set_properties_to_map(g_property_map)
        routes.append(new_route)
        
def set_manual_route_property(route,link,last_delay):
    route.properties['e_text_parallel'] = True
    route.properties['e_end_mark'] = "arrow"
    route.properties['e_pen_width'] = 2
    route.properties['e_marker_size'] = 10
    route.properties['e_color'] = "b"
    last_delay[0] += link.properties['e_up_delay']
    route.properties['e_text'] = f"Delay:{get_delay_format_micros(last_delay[0])}"
    route.properties['e_text_font_size'] = 10
    route.properties['e_text_distance'] = 10.0
    
    
def set_best_route_property(route,link,toward,last_delay):
    route.properties['e_text_parallel'] = True
    route.properties['e_end_mark'] = "arrow"
    route.properties['e_pen_width'] = 2
    route.properties['e_marker_size'] = 10
    if toward == "up":
        route.properties['e_color'] = "g"
        last_delay[0] += link.properties['e_up_delay']
    else:
        route.properties['e_color'] = "r"
        last_delay[0] += link.properties['e_down_delay']
    route.properties['e_text'] = f"Delay:{get_delay_format_micros(last_delay[0])}"
    route.properties['e_text_font_size'] = 10
    route.properties['e_text_distance'] = 10.0

# Emphasize the visited links
def set_link_property_used(link):
    for link in links:
        link.properties['e_text'] = f'{link.get_destance_format()} {link.get_band_format()}'
        if link.is_used == 1:
            link.properties['e_color'] = "black"
            
def clear_all_route(g,routes,links):
    for route in routes:
        g.remove_edge(route.edge)
    routes.clear()
    for link in links:
        link.is_used = 0

In [11]:
# set default_properties for all vertex and edge
def set_default_properties(links,nodes):
    for link in links:
        set_link_property_basic(link)
        set_link_property_used(link)     
        link.set_properties_to_map(g_property_map)
    for node in nodes:
        set_node_property(node)
        node.set_properties_to_map(g_property_map)

In [12]:
# main generate part
mode = config.get('config', 'mode')
if mode == "manual":
    create_node_manual()
    create_link_manual()
    create_routes_manual()
elif mode == "auto":
    create_node_auto()
    create_link_auto()
    g_pos = sfdp_layout(g,p=3.0,groups=g_property_map['v_group'], gamma=5.0, K=300, mu=500,kappa=1.0)
    
set_default_properties(links,nodes)

In [13]:
# output the shortest result
def out_put_result():
    print(f'Total Package Size = {package_size}Byte')
    print(f"The route with least delay shows as follow:")
    route_num = len(route_nodes_list) 
    total_delay = 0.0
    for node_pair in route_nodes_list[route_num//2:]+route_nodes_list[:route_num//2]:
        start_node = find_node_by_name(nodes,node_pair[0])
        end_node = find_node_by_name(nodes,node_pair[1])
        link = find_link_by_nodes(links, start_node, end_node)
        delay = 0.0
        if node_pair[2] == "up":
            delay = link.properties['e_up_delay']
        else:
            delay = link.properties['e_down_delay']
        total_delay += delay
        print(f"{start_node.show_name} -> {end_node.show_name}."
              f"\nDelay = {get_delay_format_micros(delay)}, Total Delay = {get_delay_format_micros(total_delay)}\n")
        print(f'Total Delay:{get_delay_format_micros(total_delay)}')

In [4]:
# print the graph
def print_g_route(output_path=""):
    a = graph_draw(g
                   , pos=g_pos
                   , vertex_text=g_property_map['v_label']
                   , output_size=(1500, 1500)
                   , bg_color="white"
                   , vertex_surface=g_property_map['sfcs']
                   , vertex_size=g_property_map['v_size']
                   , vertex_color=g_property_map['v_color']
                   , vertex_fill_color=g_property_map['v_fill_color']
                   , vertex_halo=g_property_map['v_halo']
                   , vertex_font_size=g_property_map['v_font_size']
                   , vertex_halo_size=g_property_map['v_halo_size']
                   , vertex_text_offset=g_property_map['v_text_offset']
                   , vertex_text_position=g_property_map['v_text_position']
                   , vertex_pen_width=g_property_map['v_pen_width']
                   , edge_end_marker=g_property_map['e_end_mark']
                   , edge_pen_width=g_property_map['e_pen_width']
                   , edge_marker_size=g_property_map['e_marker_size']
                   , edge_text=g_property_map['e_text']
                   , edge_text_parallel=g_property_map['e_text_parallel']
                   , edge_font_size=g_property_map['e_text_font_size']
                   , edge_color=g_property_map['e_color']
                   , edge_text_distance=g_property_map['e_text_distance']
                   , output=output_path if output_path != "" else None
                   )

# print_g_route()

In [16]:
# generate the csv file
def generate_all_link_info(step=0):
    csv_output = [["index", "distance(Km)", "bandwidth(MB)", "speed(10^8m/s)", "loss rate(%)", "delay(Millisecond)"]]

    for i, link in enumerate(links):
        csv_output.append([f"{2*i}", link.prop_distance, link.trans_bandwidth, link.propagation_speed, link.loss_up, link.properties['e_up_delay']])
        csv_output.append([f"{2*i + 1}", link.prop_distance, link.trans_bandwidth, link.propagation_speed, link.loss_down, link.properties['e_down_delay']])

    write_csv(f"output/link_info/link_info_{step}.csv", csv_output)


In [17]:
# get the delay between every two clocks
def get_all_clocks_delay(step=0):
    data=[]
    dj_clock_nodes = [dj_node for dj_node in dj_nodes if dj_node.type == 'Clock']
    data.append([dj_clock_node.show_name for dj_clock_node in dj_clock_nodes])
    data.append([find_node_by_name(nodes,dj_clock_node.name).time for dj_clock_node in dj_clock_nodes])
    for dj_clock_node_start in dj_clock_nodes:
        dist, pred = dijkstra_search(dj_g, dj_g_e_delay, dj_clock_node_start.vertex)
        for dj_clock_node_end in dj_clock_nodes:
            delay = 0.0
            node = dj_clock_node_end
            while node != dj_clock_node_start:
                pre_node = find_node_by_vertex(dj_nodes,dj_g.vertex(pred[node.vertex]))
                if(node == pre_node):
                    break
                link = find_link_by_nodes_name(dj_links,pre_node.show_name,node.show_name)
                if (link is not None):
                    delay+=link.delay
                node = pre_node
            data.append([dj_clock_node_start.show_name,dj_clock_node_end.show_name,delay])
    
    return data


def save_now_clocks_delay(step=0):
    write_csv(f"output/clocks_delay/clocks_delay_{step}.csv", get_all_clocks_delay(step))

def save_all_links_delay(links,step=0):
    for link in links:
        link.set_properties_to_map(g_property_map)
    generate_all_link_info(step)
    save_now_clocks_delay(step)